In [ ]:
### Import required libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

import os

In [ ]:
### Make sure that 'ggplot' style is used for all plots
plt.style.use('ggplot')
# plt.style.available ### To view all other available styles

In [ ]:
### Set Working Directory (WD)
os.chdir('/Volumes/GoogleDrive/My Drive/CEMEX/Data Translators/GitHub/rgamerosl/capstone-project')

In [ ]:
### Read the data
df = pd.read_excel("dataset/data_v0.xlsx")

In [ ]:
df.info()

In [ ]:
df.iloc[-1,]

In [ ]:
### For some strange reason when reading the Excel file it detects 1'021,336 entries. However I know there are only 466,786 differente entries, everything else are just empty rows
df = df.iloc[0:466786,0:18]
df.info()

In [ ]:
df_backup = df.copy(deep=True)

In [ ]:
display(df.head(7))

### Considering that "liters_per_hour" is simple the division between "Fuel_used" and "Engine_hrs" then I should exclude these two columns from my model
### Also as I see it the "Drive_time" is only transform the "Engine_hrs" into hh:mm:ss format, therefore it should also be ignored.

In [ ]:
df['liters_per_hour'].describe()

In [ ]:
### First replace empty strings with nan
df['liters_per_hour'] = df['liters_per_hour'].replace(r'^\s*$', np.nan, regex=True)
### Transform strings into float numbers
df['liters_per_hour'] = df['liters_per_hour'].astype(float)

In [ ]:
df['liters_per_hour'].describe()

In [ ]:
plt.hist(df['liters_per_hour'])
plt.show()

In [ ]:
### Save dataset with outliers for liters_per_hour
outliers1 = df.loc[abs(df['liters_per_hour']) > 20,]
outliers2 = df.loc[df['liters_per_hour'] < 0,]
outliers = pd.concat([outliers1, outliers2])
# outliers.to_excel("dataset/outliers_lts.xlsx")

In [ ]:
### Save dataset with outliers for Engine_hrs and Idle_time
outliers_1 = df.loc[abs(df['Engine_hrs']) > 24,]
outliers_2 = df.loc[abs(df["Idle_time"]) > 1,]
outliers_ = pd.concat([outliers_1, outliers_2])
# outliers_.to_excel("dataset/outliers_hrs.xlsx")

In [ ]:
### Everything bigger than 20 should be nan
df.loc[abs(df['liters_per_hour']) > 20,'liters_per_hour'] = np.nan
### Also every negative value should be nan
df.loc[df['liters_per_hour'] < 0,'liters_per_hour'] = np.nan

In [ ]:
plt.hist(df['liters_per_hour'])
plt.show()

In [ ]:
df['liters_per_hour'].describe()

In [ ]:
### Amount of nan in the liters_per_hour (target value)
df['liters_per_hour'].isna().sum()/len(df['liters_per_hour'])

In [ ]:
df['Manufacturer'].unique()

In [ ]:
### Find out how many observations we have from each manufacturer
df.Manufacturer.value_counts()

In [ ]:
### Group different spellings for the same manufacturer

df.loc[df['Manufacturer']=="INTERNATIONAL","Manufacturer"] = "International"
df.loc[df['Manufacturer']=="FREIGHTLINER","Manufacturer"] = "Freightliner"
df.loc[df['Manufacturer']=="freightliner","Manufacturer"] = "Freightliner"
df.loc[df['Manufacturer']=="MAN","Manufacturer"] = "Man"

In [ ]:
### Count the amount of differents trucks analysed by Manufacturers
print(len(df_backup['Plate'].unique()))
df.groupby('Manufacturer').Plate.nunique() 

### Something could be wrong as there are only 2330 different Plates numbers, however when doing the counts by manufacturer I get 2592 Plates.

In [ ]:
Plates = df.groupby('Plate').Manufacturer.nunique()
display(Plates.head(10))

In [ ]:
len(Plates[Plates > 1])
display(Plates[Plates > 1])

# Plates[Plates > 1].to_excel("dataset/CRs_with_2_Manufacturers.xlsx")

### There are 262 Plates numbers that have listed 2 different manufacturers, what should we do with those?
### Maybe look for an extra dataset only with the Plate number and Manufacturer to validate the real value for the Manufacturer of these 262 Trucks

In [ ]:
CRs = df_backup[df_backup["Plate"].isin(Plates[Plates > 1].index.values)]
len(CRs['Plate'].unique())

# CRs.to_excel("dataset/CRs_multiple_manufacturers.xlsx")

In [ ]:
df.loc[df["Plate"]=="CR2856","Manufacturer"].unique()

In [ ]:
### Considering that there are only 1 truck from the following Manufactrers: Astra, Scania and Volvo, I think we could get rid of these 3 trucks for the analysis
df2 = df[~df.Manufacturer.isin(["Astra", "Scania", "Volvo"])]
print(df2.Manufacturer.value_counts())
print(df2.groupby('Manufacturer').Plate.nunique())

In [ ]:
df2.info()

In [ ]:
df2.loc[0,'Date']

In [ ]:
### Add column for Weekdays (0: Monday to 6: Sunday)
df2['Weekday'] = pd.to_datetime(df2['Date']).dt.dayofweek

In [ ]:
df2.loc[1,]

In [ ]:
df2['Weekday'].unique()

In [ ]:
df2.Weekday.value_counts().sort_index()

In [ ]:
df2['Idle_time'].describe()
### It looks to me like Idle_time should always be a value between 0 and 1, so maybe everything bigger than 1 could be a typo (should it be divided by 24 maybe?)

In [ ]:
df2['TurnOn'].describe()
### Higher values seems very unlikely, what to do with them?

In [ ]:
df2['Engine_hrs'].describe()
### How could you have Engine_hrs bigger than 24 in just one day?